In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.feature_processing import *
from src.unit_proccessing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = UnitDataProcessing(config)
self = features_class

In [ ]:
df_item = features_class.df_item.copy()
df_unit = features_class.df_unit.copy()

In [ ]:
feature_name = 'f__numeric_response'
score_name = 's__numeric_response'""
df = df_item[~pd.isnull(df_item[feature_name])].copy()

valid_variables = df.groupby('variable_name').filter(lambda group: len(group[feature_name].unique()) >= 3 and len(group) > 100)

# Get the unique variable names that meet the conditions
variables = valid_variables['variable_name'].unique()

#pivot_table, index_col = self.get_clean_pivot_table(feature_name,remove_low_freq_col=True)

In [ ]:
for index_range in range(0, len(variables), 50):
    plt.figure(figsize=(15, 6))
    sns.boxplot(df[df['variable_name'].isin(variables[index_range:index_range+50])],x='variable_name', y=feature_name)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
from scipy.stats import ks_2samp

def km_test(dist1, dist2, minimum_sample = 50):
    score = None
    if dist1.shape[0] >= minimum_sample and dist2.shape[0] >= minimum_sample: 
        stat, p = ks_2samp(dist1, dist2)
        score = 0
        alpha = 0.05
        if p < alpha:
            score = 1
    return score

In [ ]:
for var in variables:
    mask1 = (df['variable_name']==var) 
    for resp in df['responsible'].unique():
        mask2 = (df['responsible']==resp)
        df.loc[mask1 & mask2,score_name] = km_test(df[mask1 & mask2][feature_name], df[mask1 & (~mask2)][feature_name])

In [ ]:
data = df.groupby(['responsible','variable_name']).agg({score_name: 'mean'})
data = data.reset_index()
entropy_ = data.groupby('responsible')[score_name].mean()
entropy_.plot(kind='bar')
plt.title('entropy__answer_position')
plt.show()

In [ ]:
resp = 'SlahF_int'
var = 'time_collect_water'
mask1 = (df['variable_name']==var) 
mask2 = (df['responsible']==resp)

bins = np.histogram_bin_edges(df[mask1][feature_name], bins=4)
no_anomaly_df = df[mask1 & mask2][feature_name]
anomaly_df = df[mask1 & (~mask2)][feature_name]
fig,  ax = plt.subplots(1,2)
ax[0].hist(no_anomaly_df, bins=bins, alpha=0.5, color='blue', label='True')
ax[1].hist(anomaly_df, bins=bins, alpha=0.5, color='red', label='False')
plt.title(var)
plt.show()

In [ ]:
for var in variables:
    mask = (df['variable_name']==var) 
    mask1 = (df[feature_name]!=0) 
    model = ECOD()
    df.loc[mask,score_name] = model.fit_predict(df[mask][[feature_name]])
    if df[(df[feature_name]==0)&mask][score_name].sum()>0:
        print(df[mask][feature_name].value_counts())
    bins = np.histogram_bin_edges(df[mask&mask1][feature_name], bins=12)
    no_anomaly_df = df[(df[score_name]==0)&mask&mask1][feature_name]
    anomaly_df = df[(df[score_name]==1)&mask&mask1][feature_name]
    
    plt.hist(no_anomaly_df, bins=bins, alpha=0.5, color='blue', label='True')
    plt.hist(anomaly_df, bins=bins, alpha=0.5, color='red', label='False')
    plt.title(var)
    plt.show()

In [ ]:

from pyod.models.pca import PCA
score_name = feature_name.replace('f__', '__')
df_item[score_name] = 0 
df[score_name] = 0 
for col in variables:
    model = PCA()#contamination=FILTER())
    model.fit(df[df['variable_name'] == col][[feature_name]])
    prediction = model.predict(df[df['variable_name'] == col][[feature_name]])
    df.loc[df['variable_name'] == col, score_name] = prediction
    indices = df[df['variable_name'] == col].index
    df_item.loc[indices, score_name] = prediction
    
    print(col,
          'min VALUE', df_item[(df_item['variable_name']==col) & (df_item[score_name]==0)][feature_name].min(), 
          '--', df_item[(df_item['variable_name']==col)][feature_name].min(), 
          'max VALUE', df_item[(df_item['variable_name']==col) & (df_item[score_name]==0)][feature_name].max(),
          '--', df_item[(df_item['variable_name']==col)][feature_name].max())


In [ ]:
df[df['variable_name'] == 'nr_hh_assets_own'][[feature_name,score_name]]

In [ ]:
df_item[df_item['variable_name'] == 'age_adult'][feature_name].value_counts()